## 項目1 使用するPythonのバージョンを知っておく

In [10]:
!python --version

Python 3.8.5


In [11]:
import sys
print(sys.version_info)
print(sys.version)

sys.version_info(major=3, minor=8, micro=5, releaselevel='final', serial=0)
3.8.5 (default, Sep  3 2020, 21:29:08) [MSC v.1916 64 bit (AMD64)]


## 項目2 PEP8スタイルガイドに従う

以下のスタイルガイドがある。

* 空白
* 名前付け
* 式と文
* インポート

## 項目3 bytesとstrの違い

In [15]:
a = b'h\x65llo'
print(list(a))
print(a)

[104, 101, 108, 108, 111]
b'hello'


In [18]:
a = 'a\u0300 propos'
print(list(a))
print(a)

['a', '̀', ' ', 'p', 'r', 'o', 'p', 'o', 's']
à propos


Unicodeサンドイッチ（strとbytesの違いで処理に誤りがおきるので変換する）

In [21]:
def to_str(bytes_or_str):
    if isinstance(bytes_or_str, bytes):
        value = bytes_or_str.decode('utf-8')
    else:
        value = bytes_or_str
    return value # str instance

print(repr(to_str(b'foo')))
print(repr(to_str('bar')))

'foo'
'bar'


In [23]:
def to_bytes(bytes_or_str):
    if isinstance(bytes_or_str, str):
        value = bytes_or_str.encode('utf-8')
    else:
        value = bytes_or_str
    return value # bytes instance

print(repr(to_bytes(b'foo')))
print(repr(to_bytes('bar')))

b'foo'
b'bar'


その他の注意

* strとbytesでは同じように扱えない
  * str、bytes同士は+演算子で結合できる、比較、同値チェックができる
    * str、bytes間ではできないので注意

In [2]:
print(b'one', b'two')
print('one' + 'two')

b'one' b'two'
onetwo


In [3]:
b'one' + 'two'

TypeError: can't concat str to bytes

In [4]:
'one' + b'two'

TypeError: can only concatenate str (not "bytes") to str

In [6]:
assert b'red' > b'blue'
assert 'red' > 'blue'

In [7]:
assert 'red' > b'blue'

TypeError: '>' not supported between instances of 'str' and 'bytes'

In [9]:
assert b'blue' < 'red'

TypeError: '<' not supported between instances of 'bytes' and 'str'

In [10]:
print(b'foo' == 'foo')

False


In [11]:
print(b'red %s' % b'blue')
print('red %s' % 'blue')

b'red blue'
red blue


In [12]:
print(b'red %s' % 'blue')

TypeError: %b requires a bytes-like object, or an object that implements __bytes__, not 'str'

In [14]:
# bytesの__repr__によって b'blue' がそのまま表示される
print('red %s' % b'blue')

red b'blue'


ファイル書き込み（バイナリ、文字列の出力）

In [27]:
with open('chapter01/random.bin', 'w') as f:
    f.write(b'\xf1\xf2\xf3\xf4\xf5')

TypeError: write() argument must be str, not bytes

In [28]:
with open('chapter01/random.bin', 'wb') as f:
    f.write(b'\xf1\xf2\xf3\xf4\xf5')

In [29]:
with open('chapter01/random.bin', 'r') as f:
    data = f.read()

UnicodeDecodeError: 'cp932' codec can't decode byte 0xf5 in position 4: incomplete multibyte sequence

In [30]:
with open('chapter01/random.bin', 'rb') as f:
    data = f.read()
assert data == b'\xf1\xf2\xf3\xf4\xf5'

In [33]:
with open('chapter01/random.bin', 'r', encoding='cp1252') as f:
    data = f.read()

assert data == 'ñòóôõ'

## 項目4 Cスタイルフォーマット文字列とstr.formatは使わずf文字列で埋め込む

### %による方法

* Cスタイルに慣れていればそのまま使える
* フォーマットの型を揃える必要がある（順序が変わるとフォーマット側も変える必要がある）
* 辞書型で渡すことができるがコードが冗長になる

In [34]:
a = 0b10111011
b = 0xc5f
print('Binary is %d, hex is %d' % (a,b))

Binary is 187, hex is 3167


In [35]:
key = 'my_var'
value = 1.234
formatted = '%-10s = %.2f' % (key, value)
print(formatted)

my_var     = 1.23


In [37]:
key = 'my_var'
value = 1.234
formatted = '%-10s = %.2f' % (value, key)
print(formatted)

TypeError: must be real number, not str

In [39]:
key = 'my_var'
value = 1.234
formatted = '%.2f = %-10s' % (key, value)
print(formatted)

TypeError: must be real number, not str

In [1]:
pantry = [
    ('avocados', 1.25),
    ('bananas', 2.5),
    ('cherries', 15),
]
for i, (item, count) in enumerate(pantry):
    print('#%d: %-10s = %.2f' % (i, item, count))

#0: avocados   = 1.25
#1: bananas    = 2.50
#2: cherries   = 15.00


In [4]:
# 書き換える必要がある
for i, (item, count) in enumerate(pantry):
    print('#%d: %-10s = %d' % (i, item, round(count)))

#0: avocados   = 1
#1: bananas    = 2
#2: cherries   = 15


In [6]:
# 問題：nameを2回記述する必要がある
template = '%s loves food. See %s cook.'
name = 'Max'
formatted = template % (name, name)
print(formatted)

Max loves food. See Max cook.


In [11]:
# 問題：メソッドを使う場合さらに長くなる
name = 'brad'
formatted = template % (name.title(), name.title())
print(formatted)

Brad loves food. See Brad cook.


In [17]:
# 解決：辞書型で渡すと交代もできる
key = 'my_var'
value = 1.234

old_way = '%-10s = %.2f' % (key, value)

new_way = '%(key)-10s = %(value).2f' % {
    'key': key, 'value': value
}

reordered = '%(key)-10s = %(value).2f' % {
    'value': value, 'key': key
}

assert old_way == new_way == reordered

In [22]:
# 解決：辞書型だと2つ入れずに入力できる
name = 'Max'

template = '%s loves food. See %s cook.'
before = template % (name, name)

template = '%(name)s loves food. See %(name)s cook.'
after = template % {'name': name}

assert after == before

In [24]:
# 問題：変数とフォーマットをいれると冗長になる
for i, (item, count) in enumerate(pantry):
    before = '#%d: %-10s = %d' % (
        i + 1,
        item.title(),
        round(count)
    )

    after = '#%(loop)d: %(item)-10s = %(count)d' % {
        'loop': i + 1,
        'item': item.title(),
        'count': round(count)
    }

    assert before == after

In [26]:
soup = 'lentil'
formatted = 'Today\'s soup is %(soup)s.' % {'soup': soup}
print(formatted)

Today's soup is lentil.


In [28]:
menu = {
    'soup': 'lenti',
    'oyster': 'kumamoto',
    'special': 'schnitzel',
}
template = ('Today\'s soup is %(soup)s, '
            'buy one get two %(oyster)s oysters,'
            'and ourt special entree is %(special)s.')
formatted = template % menu
print(formatted)

Today's soup is lenti, buy one get two kumamoto oysters,and ourt special entree is schnitzel.


### 組み込みのformatとstr.format

* %演算子と比べると型指定せずにプレースホルダが使える利点がある
    * 型指定がないので順序変更に強い
* それ以外については記法が違うもののメリット・デメリットに大きな差はない

In [38]:
a = 1234.5678
formatted = format(a, ',.2f')
print(formatted)

b = 'my string'
formatted = format(b, '^20s')
print('*', formatted, '*')

1,234.57
*      my string       *


In [39]:
key = 'my_var'
value = 1.234

formatted = '{} = {}'.format(key, value)
print(formatted)

my_var = 1.234


In [40]:
formatted = '{:<10} = {:.2f}'.format(key, value)
print(formatted)

my_var     = 1.23


In [41]:
# エスケープ
print('%.2f%%' % 12.5)
print('{} replaces {{}}'.format(1.23))

12.50%
1.23 replaces {}


In [42]:
# プレースホルダに順序指定ができる
formatted = '{1} = {0}'.format(key, value)
print(formatted)

1.234 = my_var


In [43]:
# プレースホルダに順序をいれることで重複を省略できる
formatted = '{0} loves food. See {0} cook.'.format(name)
print(formatted)

Max loves food. See Max cook.


In [46]:
# 変更に弱い点は変わらない
for i, (item, count) in enumerate(pantry):
    old_style = '#%d: %-10s = %d' % (
        i + 1,
        item.title(),
        round(count),
    )

    new_style = '#{}: {:<10s} = {}'.format(
        i + 1,
        item.title(),
        round(count),
    )

    assert old_style == new_style


In [47]:
# 辞書のキーと組み合わせることもできるaが、冗長性の解消にはならない
formatted = 'First letter is {menu[oyster][0]!r}'.format(menu=menu)
print(formatted)

First letter is 'k'


In [52]:
# 辞書型の連携も冗長さは変わらない
menu = {
    'soup': 'lenti',
    'oyster': 'kumamoto',
    'special': 'schnitzel',
}
old_template = ('Today\'s soup is %(soup)s, '
            'buy one get two %(oyster)s oysters, '
            'and our special entree is %(special)s.')
old_formatted = old_template % menu

new_template = (
    'Today\'s soup is {soup}, '
    'buy one get two {oyster} oysters, '
    'and our special entree is {special}.'
)
new_formatted = new_template.format(**menu)

assert new_formatted == old_formatted

### フォーマット済み文字列

Python3.6以降で登場。